#### Setup

In [ ]:
!pip install -U tensorflow keras

In [ ]:
!pip install -U talos

In [ ]:
!pip install -U fasttext

In [ ]:
# Import general Python libraries
import pandas as pd
import numpy as np
import random
import sklearn
import seaborn as sns
import os
import io
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Specify seeds for random-operations
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
# Import sklearn-specific modules
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
# Import tensorflow-specific modules
import tensorflow as tf
tf.random.set_seed(seed_value)
print("Tensorflow Version: {}".format(tf.__version__))
print("Keras Version: {}".format(tf.keras.__version__))

Tensorflow Version: 2.4.1
Keras Version: 2.4.0


In [ ]:
# Import keras-specific modules
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dropout, BatchNormalization, LayerNormalization, GaussianNoise, Activation
from tensorflow.keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPool1D, AvgPool1D, GlobalMaxPool1D, GlobalAvgPool1D, RNN, GRU, LSTM, SeparableConv1D, SimpleRNN, Bidirectional, LocallyConnected1D, LeakyReLU, Input
from tensorflow.keras.optimizers import Adadelta, RMSprop, Adam, Adamax, Nadam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform, LecunNormal, LecunUniform, HeNormal, HeUniform, Constant
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.utils import plot_model
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# Import talos-specific modules
import talos

In [ ]:
# Set pandas options
pd.set_option("display.max_columns", None)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set up TPU configurations
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.52.181.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.52.181.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


#### Import Dataset:

In [ ]:
# Import Kickstarter Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Text.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 7)
246891


,campaign_successful,title,blurb,story,risks,reward_description,creator_bio
22821161,0,sentio golf putters. feel is the difference,choose the feel you want with our patented flo...,sentio putters feature a unique floating face...,high tech process although we have made severa...,our eternal gratitude. every little bit helps ...,sentio golf is driven to produce the most adva...


In [ ]:
# Import Word Embeddings
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
print(ft.get_dimension())

300


#### a) Baseline

This model provides the baseline, which will be used as starting point for the hyperparameter search. The goal is to gradually improve the model performance.

**Hyperparameters:**
- Preprocessing: Only Story, Removed Stopwords, Max_Features = No Restrictions, Max_Len = 500, Embedding_Dim = 300, Fasttext Embeddings
- CNN: 2 Conv Layers (64 Filters, Window Size 7, ReLU, padding=same), MaxPooling (Size 5), GlobalMaxPooling, Dropout before Dense Classifier (Rate = 0.5), 1 Hidden Dense Layer (32 Units, ReLU), Nadam, Batch Size 512
- RNN: 1 LSTM Layer (32 Units), RMSProp, Batch Size 512


##### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["story"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

100%|██████████| 246891/246891 [00:23<00:00, 10588.88it/s]


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
# Define parameters for text processing 
max_features = 471366
max_len = 500
embedding_dim = 300

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(tokenizer.num_words))

Included Token: 471366


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,243.835115
std,248.457559
min,0.000000
25%,90.000000
50%,167.000000
75%,310.000000
max,3555.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 500)
Shape of X_val: (37034, 500)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (471366, 300)
Number of Null Word Embeddings: 152


##### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 300)          141409800 
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 64)           134464    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 64)           28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)               

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 21s 45ms/step - loss: 0.5949 - binary_accuracy: 0.6845 - val_loss: 0.5221 - val_binary_accuracy: 0.7383
Epoch 2/100
338/338 [==============================] - 10s 29ms/step - loss: 0.5273 - binary_accuracy: 0.7360 - val_loss: 0.5415 - val_binary_accuracy: 0.7265
Epoch 3/100
338/338 [==============================] - 10s 29ms/step - loss: 0.5062 - binary_accuracy: 0.7529 - val_loss: 0.5043 - val_binary_accuracy: 0.7510
Epoch 4/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4917 - binary_accuracy: 0.7615 - val_loss: 0.4980 - val_binary_accuracy: 0.7540
Epoch 5/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4779 - binary_accuracy: 0.7707 - val_loss: 0.4970 - val_binary_accuracy: 0.7538
Epoch 6/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4593 - binary_accuracy: 0.7822 - val_loss: 0.5066 - val_binary_accuracy: 0.7559
Epoch 7/100
338/338 [=========================

##### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          141409800 
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 141,452,457
Trainable params: 42,657
Non-trainable params: 141,409,800
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 25s 63ms/step - loss: 0.6033 - binary_accuracy: 0.6736 - val_loss: 0.5529 - val_binary_accuracy: 0.7200
Epoch 2/100
338/338 [==============================] - 17s 49ms/step - loss: 0.5540 - binary_accuracy: 0.7201 - val_loss: 0.5553 - val_binary_accuracy: 0.7088
Epoch 3/100
338/338 [==============================] - 16s 49ms/step - loss: 0.5444 - binary_accuracy: 0.7287 - val_loss: 0.5326 - val_binary_accuracy: 0.7336
Epoch 4/100
338/338 [==============================] - 16s 49ms/step - loss: 0.5425 - binary_accuracy: 0.7273 - val_loss: 0.5280 - val_binary_accuracy: 0.7367
Epoch 5/100
338/338 [==============================] - 17s 49ms/step - loss: 0.5361 - binary_accuracy: 0.7318 - val_loss: 0.5251 - val_binary_accuracy: 0.7383
Epoch 6/100
338/338 [==============================] - 17s 49ms/step - loss: 0.5345 - binary_accuracy: 0.7338 - val_loss: 0.5233 - val_binary_accuracy: 0.7416
Epoch 7/100
338/338 [=========================

#### b) Test Different Preprocessing Strategies

##### Check How Different Paddings Affect Model Performance:

###### Data Preparation:

In [ ]:
# Define parameters for text processing 
max_features = 471366
max_len = 500
embedding_dim = 300

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(max_features))

Included Token: 471366


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,243.835115
std,248.457559
min,0.000000
25%,90.000000
50%,167.000000
75%,310.000000
max,3555.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="post", truncating="pre")
X_val = pad_sequences(X_val, maxlen=max_len, padding="post", truncating="pre")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 500)
Shape of X_val: (37034, 500)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (471366, 300)
Number of Null Word Embeddings: 152


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 300)          141409800 
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 500, 64)           134464    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 100, 64)           28736     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)               

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 21s 45ms/step - loss: 0.5937 - binary_accuracy: 0.6854 - val_loss: 0.5255 - val_binary_accuracy: 0.7361
Epoch 2/100
338/338 [==============================] - 10s 29ms/step - loss: 0.5283 - binary_accuracy: 0.7355 - val_loss: 0.5574 - val_binary_accuracy: 0.7134
Epoch 3/100
338/338 [==============================] - 10s 29ms/step - loss: 0.5102 - binary_accuracy: 0.7482 - val_loss: 0.5045 - val_binary_accuracy: 0.7507
Epoch 4/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4930 - binary_accuracy: 0.7602 - val_loss: 0.4985 - val_binary_accuracy: 0.7547
Epoch 5/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4806 - binary_accuracy: 0.7692 - val_loss: 0.5001 - val_binary_accuracy: 0.7524
Epoch 6/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4616 - binary_accuracy: 0.7792 - val_loss: 0.5052 - val_binary_accuracy: 0.7573
Epoch 7/100
338/338 [=========================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 300)          141409800 
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 141,452,457
Trainable params: 42,657
Non-trainable params: 141,409,800
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 25s 62ms/step - loss: 0.6466 - binary_accuracy: 0.6520 - val_loss: 0.6101 - val_binary_accuracy: 0.6699
Epoch 2/100
338/338 [==============================] - 17s 50ms/step - loss: 0.6222 - binary_accuracy: 0.6447 - val_loss: 0.6378 - val_binary_accuracy: 0.6518
Epoch 3/100
338/338 [==============================] - 17s 49ms/step - loss: 0.6381 - binary_accuracy: 0.6550 - val_loss: 0.6219 - val_binary_accuracy: 0.6570
Epoch 4/100
338/338 [==============================] - 16s 49ms/step - loss: 0.6331 - binary_accuracy: 0.6551 - val_loss: 0.7177 - val_binary_accuracy: 0.6576
Epoch 5/100
338/338 [==============================] - 17s 49ms/step - loss: 0.6287 - binary_accuracy: 0.6560 - val_loss: 0.6174 - val_binary_accuracy: 0.6574
Epoch 6/100
338/338 [==============================] - 17s 49ms/step - loss: 0.6196 - binary_accuracy: 0.6576 - val_loss: 0.6119 - val_binary_accuracy: 0.6442
Training Accuracy: 0.670
Validation Accuracy: 

###### Result:

Padding / Truncating:
- Pre/Pre: CNN = 80.6% vs. 75.7%; RNN = 76.4% vs. 75.6%
- Post/Post: CNN = 80.6% vs. 75.5%; 68.1% vs. 67.8%
- Pre/Post: CNN = 81.8% vs. 75.6%; RNN = 76.9% vs. 75.8%
- Post/Pre: CNN = 80.5% vs. 75.7%; RNN = 67% vs. 67%

Conclusion:
- RNN was not able to learn when padding="post" (probably because short sentences are processed sequentially and at the end many null-embeddings are considered)
- CNN was relatively unaffected, as it just searches for local cues
- Padding="pre" & Truncating="post" provided the best results

##### Check how different embedding dimensions affect model performance:

###### Data Preparation:

In [ ]:
# Reduce embedding dimension
print("Before: {}".format(ft.get_dimension()))
fasttext.util.reduce_model(ft, 200)
print("After: {}".format(ft.get_dimension()))

Before: 300
After: 200


In [ ]:
# Define parameters for text processing 
max_features = 471366
max_len = 500
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(max_features))

Included Token: 471366


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,243.835115
std,248.457559
min,0.000000
25%,90.000000
50%,167.000000
75%,310.000000
max,3555.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 500)
Shape of X_val: (37034, 500)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (471366, 200)
Number of Null Word Embeddings: 152


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 200)          94273200  
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 500, 64)           89664     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 100, 64)           28736     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 64)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 32)              

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 22s 51ms/step - loss: 0.6044 - binary_accuracy: 0.6743 - val_loss: 0.5361 - val_binary_accuracy: 0.7309
Epoch 2/100
338/338 [==============================] - 14s 41ms/step - loss: 0.5444 - binary_accuracy: 0.7231 - val_loss: 0.5508 - val_binary_accuracy: 0.7082
Epoch 3/100
338/338 [==============================] - 31s 92ms/step - loss: 0.5262 - binary_accuracy: 0.7356 - val_loss: 0.5151 - val_binary_accuracy: 0.7460
Epoch 4/100
338/338 [==============================] - 9s 26ms/step - loss: 0.5144 - binary_accuracy: 0.7444 - val_loss: 0.5129 - val_binary_accuracy: 0.7427
Epoch 5/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5041 - binary_accuracy: 0.7526 - val_loss: 0.5061 - val_binary_accuracy: 0.7488
Epoch 6/100
338/338 [==============================] - 9s 26ms/step - loss: 0.4931 - binary_accuracy: 0.7583 - val_loss: 0.5077 - val_binary_accuracy: 0.7477
Epoch 7/100
338/338 [============================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 200)          94273200  
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                29824     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 33        
Total params: 94,303,057
Trainable params: 29,857
Non-trainable params: 94,273,200
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 30s 78ms/step - loss: 0.6120 - binary_accuracy: 0.6698 - val_loss: 0.5548 - val_binary_accuracy: 0.7127
Epoch 2/100
338/338 [==============================] - 15s 46ms/step - loss: 0.5592 - binary_accuracy: 0.7139 - val_loss: 0.5617 - val_binary_accuracy: 0.7174
Epoch 3/100
338/338 [==============================] - 15s 46ms/step - loss: 0.5520 - binary_accuracy: 0.7216 - val_loss: 0.5391 - val_binary_accuracy: 0.7275
Epoch 4/100
338/338 [==============================] - 15s 45ms/step - loss: 0.5492 - binary_accuracy: 0.7218 - val_loss: 0.5355 - val_binary_accuracy: 0.7304
Epoch 5/100
338/338 [==============================] - 16s 46ms/step - loss: 0.5442 - binary_accuracy: 0.7250 - val_loss: 0.5410 - val_binary_accuracy: 0.7229
Epoch 6/100
338/338 [==============================] - 16s 46ms/step - loss: 0.5408 - binary_accuracy: 0.7242 - val_loss: 0.5371 - val_binary_accuracy: 0.7310
Epoch 7/100
338/338 [=========================

###### Results:

- 300 dimension: CNN = 79.5% vs. 75.7%; RNN = 77.2% vs. 75.8%
- 200 dimensions: CNN = 77.4% vs. 74.9% ; RNN = 77.7% vs. 76.2%
- 100 dimensions: CNN = 78.3% vs. 74.8%; RNN = 72.7% vs. 72.6%
- 50 dimensions: CNN = 77.7% vs. 74.5%; RNN = 73.8% vs. 73.6%
- 200 dimensions worked especially well with RNNs; 300 dimensions worked better with CNNs (i.e. stick with 200 dimensions for now)

##### Check if self-trained word embeddings help:

###### Data Preparation:

In [ ]:
# Define parameters for text processing 
max_features = 100000
max_len = 500
embedding_dim = 50

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(tokenizer.num_words))

Included Token: 100000


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,240.113306
std,243.629157
min,0.000000
25%,88.000000
50%,165.000000
75%,305.000000
max,2810.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 500)
Shape of X_val: (37034, 500)


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, input_length=max_len))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Dropout(0.5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 500, 50)           5000000   
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 500, 64)           22464     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 100, 64)           0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 100, 64)           28736     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 64)                0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)              

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 24s 50ms/step - loss: 0.5947 - binary_accuracy: 0.6839 - val_loss: 0.5109 - val_binary_accuracy: 0.7555
Epoch 2/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4808 - binary_accuracy: 0.7712 - val_loss: 0.5046 - val_binary_accuracy: 0.7514
Epoch 3/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4494 - binary_accuracy: 0.7910 - val_loss: 0.4962 - val_binary_accuracy: 0.7601
Epoch 4/100
338/338 [==============================] - 10s 30ms/step - loss: 0.4232 - binary_accuracy: 0.8079 - val_loss: 0.5014 - val_binary_accuracy: 0.7552
Epoch 5/100
338/338 [==============================] - 10s 31ms/step - loss: 0.3943 - binary_accuracy: 0.8261 - val_loss: 0.5225 - val_binary_accuracy: 0.7421
Epoch 6/100
338/338 [==============================] - 10s 31ms/step - loss: 0.3624 - binary_accuracy: 0.8439 - val_loss: 0.5269 - val_binary_accuracy: 0.7435
Epoch 7/100
338/338 [=========================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, input_length=max_len))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 500, 50)           5000000   
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                10624     
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 33        
Total params: 5,010,657
Trainable params: 5,010,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 31s 69ms/step - loss: 0.5743 - binary_accuracy: 0.7006 - val_loss: 0.5065 - val_binary_accuracy: 0.7588
Epoch 2/100
338/338 [==============================] - 17s 49ms/step - loss: 0.4775 - binary_accuracy: 0.7783 - val_loss: 0.5671 - val_binary_accuracy: 0.7508
Epoch 3/100
338/338 [==============================] - 17s 51ms/step - loss: 0.4487 - binary_accuracy: 0.7965 - val_loss: 0.5154 - val_binary_accuracy: 0.7587
Epoch 4/100
338/338 [==============================] - 17s 50ms/step - loss: 0.4268 - binary_accuracy: 0.8089 - val_loss: 0.5223 - val_binary_accuracy: 0.7411
Epoch 5/100
 15/338 [>.............................] - ETA: 14s - loss: 0.4022 - binary_accuracy: 0.8285

###### Results:

- 100,000 tokens; 100 dimensions: CNN = 78.6% vs. 75.9%; RNN = 81.8% vs. 75.7%
- 50,000 tokens; 100 dimensions: CNN = 78.8% vs. 75.8%; RNN = 78% vs. 75.7%
- 100,000 tokens, 50 dimensions: CNN = 82% vs. 76%; RNN = 79.7% vs. 75.9%
- using own word embeddings leads to strong overfitting, which also cannot be mitigated by using dropout

##### Initialize Embeddings with Pre-Trained Weights and Fine-Tune Them:

###### Data Preparation:

In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.100.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 100


In [ ]:
# Define parameters for text processing 
max_features = 100000
max_len = 500
embedding_dim = 100

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(tokenizer.num_words))

Included Token: 100000


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,240.113306
std,243.629157
min,0.000000
25%,88.000000
50%,165.000000
75%,305.000000
max,2810.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 500)
Shape of X_val: (37034, 500)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (100000, 100)
Number of Null Word Embeddings: 15


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 100)          10000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 64)           44864     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 100, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 64)           28736     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 21s 47ms/step - loss: 0.5822 - binary_accuracy: 0.6939 - val_loss: 0.4964 - val_binary_accuracy: 0.7604
Epoch 2/100
338/338 [==============================] - 9s 27ms/step - loss: 0.4768 - binary_accuracy: 0.7732 - val_loss: 0.5198 - val_binary_accuracy: 0.7442
Epoch 3/100
338/338 [==============================] - 9s 28ms/step - loss: 0.4417 - binary_accuracy: 0.7954 - val_loss: 0.4894 - val_binary_accuracy: 0.7641
Epoch 4/100
338/338 [==============================] - 9s 28ms/step - loss: 0.4043 - binary_accuracy: 0.8171 - val_loss: 0.5013 - val_binary_accuracy: 0.7596
Epoch 5/100
338/338 [==============================] - 10s 28ms/step - loss: 0.3508 - binary_accuracy: 0.8491 - val_loss: 0.5530 - val_binary_accuracy: 0.7587
Epoch 6/100
338/338 [==============================] - 9s 28ms/step - loss: 0.2829 - binary_accuracy: 0.8828 - val_loss: 0.6450 - val_binary_accuracy: 0.7524
Epoch 7/100
338/338 [=============================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          10000000  
_________________________________________________________________
lstm (LSTM)                  (None, 32)                17024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 10,017,057
Trainable params: 10,017,057
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 29s 67ms/step - loss: 0.5802 - binary_accuracy: 0.6938 - val_loss: 0.5074 - val_binary_accuracy: 0.7547
Epoch 2/100
338/338 [==============================] - 16s 47ms/step - loss: 0.4842 - binary_accuracy: 0.7732 - val_loss: 0.5083 - val_binary_accuracy: 0.7518
Epoch 3/100
338/338 [==============================] - 16s 49ms/step - loss: 0.4531 - binary_accuracy: 0.7926 - val_loss: 0.5037 - val_binary_accuracy: 0.7551
Epoch 4/100
338/338 [==============================] - 16s 48ms/step - loss: 0.4262 - binary_accuracy: 0.8088 - val_loss: 0.5102 - val_binary_accuracy: 0.7506
Epoch 5/100
338/338 [==============================] - 16s 48ms/step - loss: 0.3962 - binary_accuracy: 0.8247 - val_loss: 0.5331 - val_binary_accuracy: 0.7497
Epoch 6/100
338/338 [==============================] - 16s 48ms/step - loss: 0.3663 - binary_accuracy: 0.8402 - val_loss: 0.5936 - val_binary_accuracy: 0.7372
Epoch 7/100
338/338 [=========================

###### Results:

- using pre-trained word embeddings and keeping them fixed leads to way less overfitting with comparable results
- i.e. keep pre-trained embeddings

##### Check how stopwords affect model performance:

###### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["story"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.200.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 200


In [ ]:
# Define parameters for text processing 
max_features = 431789
max_len = 700
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(tokenizer.num_words))

Included Token: None


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,506.934945
std,499.431631
min,0.000000
25%,194.000000
50%,355.000000
75%,646.000000
max,6766.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 700)
Shape of X_val: (37034, 700)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (431789, 200)
Number of Null Word Embeddings: 157


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 700, 200)          86357800  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 700, 64)           89664     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 140, 64)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 140, 64)           28736     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)               

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 20s 45ms/step - loss: 0.6209 - binary_accuracy: 0.6635 - val_loss: 0.5612 - val_binary_accuracy: 0.7074
Epoch 2/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5667 - binary_accuracy: 0.7029 - val_loss: 0.5621 - val_binary_accuracy: 0.6964
Epoch 3/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5489 - binary_accuracy: 0.7182 - val_loss: 0.5459 - val_binary_accuracy: 0.7317
Epoch 4/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5389 - binary_accuracy: 0.7260 - val_loss: 0.5353 - val_binary_accuracy: 0.7378
Epoch 5/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5307 - binary_accuracy: 0.7325 - val_loss: 0.5256 - val_binary_accuracy: 0.7362
Epoch 6/100
338/338 [==============================] - 9s 28ms/step - loss: 0.5233 - binary_accuracy: 0.7361 - val_loss: 0.5207 - val_binary_accuracy: 0.7397
Epoch 7/100
338/338 [==============================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 700, 200)          86357800  
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                29824     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 86,387,657
Trainable params: 29,857
Non-trainable params: 86,357,800
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 27s 67ms/step - loss: 0.6338 - binary_accuracy: 0.6489 - val_loss: 0.5916 - val_binary_accuracy: 0.6805
Epoch 2/100
338/338 [==============================] - 18s 55ms/step - loss: 0.6076 - binary_accuracy: 0.6705 - val_loss: 0.6131 - val_binary_accuracy: 0.6947
Epoch 3/100
338/338 [==============================] - 18s 55ms/step - loss: 0.5943 - binary_accuracy: 0.6814 - val_loss: 0.5771 - val_binary_accuracy: 0.7001
Epoch 4/100
338/338 [==============================] - 18s 54ms/step - loss: 0.5868 - binary_accuracy: 0.6893 - val_loss: 0.5704 - val_binary_accuracy: 0.7019
Epoch 5/100
338/338 [==============================] - 19s 56ms/step - loss: 0.5771 - binary_accuracy: 0.7007 - val_loss: 0.6457 - val_binary_accuracy: 0.6651
Epoch 6/100
338/338 [==============================] - 19s 55ms/step - loss: 0.5802 - binary_accuracy: 0.6933 - val_loss: 0.5573 - val_binary_accuracy: 0.7176
Epoch 7/100
338/338 [=========================

###### Results:

- Stopwords Not Removed: CNN = 78.2% vs. 74.4%; RNN = 72.7%
- i.e. NNs struggled when stopwords were not removed

##### Check if adding additional text features can increase model performance:

###### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

100%|██████████| 246891/246891 [00:42<00:00, 5815.11it/s]


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.200.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 200


In [ ]:
# Define parameters for text processing 
max_features = 631377
max_len = 500
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(len(word_index)))

Included Token: 631377


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,509.529984
std,409.239714
min,14.000000
25%,241.000000
50%,399.000000
75%,649.000000
max,11553.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 500)
Shape of X_val: (37034, 500)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (631377, 200)
Number of Null Word Embeddings: 218


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 200)          126275400 
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 64)           89664     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 100, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 64)           28736     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 19s 44ms/step - loss: 0.6020 - binary_accuracy: 0.6732 - val_loss: 0.5266 - val_binary_accuracy: 0.7350
Epoch 2/100
338/338 [==============================] - 9s 28ms/step - loss: 0.5335 - binary_accuracy: 0.7296 - val_loss: 0.5457 - val_binary_accuracy: 0.7190
Epoch 3/100
338/338 [==============================] - 10s 28ms/step - loss: 0.5154 - binary_accuracy: 0.7422 - val_loss: 0.5039 - val_binary_accuracy: 0.7557
Epoch 4/100
338/338 [==============================] - 9s 28ms/step - loss: 0.5040 - binary_accuracy: 0.7519 - val_loss: 0.5097 - val_binary_accuracy: 0.7467
Epoch 5/100
338/338 [==============================] - 10s 28ms/step - loss: 0.4928 - binary_accuracy: 0.7581 - val_loss: 0.4898 - val_binary_accuracy: 0.7611
Epoch 6/100
338/338 [==============================] - 9s 28ms/step - loss: 0.4780 - binary_accuracy: 0.7681 - val_loss: 0.4884 - val_binary_accuracy: 0.7634
Epoch 7/100
338/338 [============================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 200)          126275400 
_________________________________________________________________
lstm (LSTM)                  (None, 32)                29824     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 126,305,257
Trainable params: 29,857
Non-trainable params: 126,275,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 23s 58ms/step - loss: 0.6131 - binary_accuracy: 0.6608 - val_loss: 0.6127 - val_binary_accuracy: 0.6198
Epoch 2/100
338/338 [==============================] - 16s 49ms/step - loss: 0.5699 - binary_accuracy: 0.7002 - val_loss: 0.5554 - val_binary_accuracy: 0.7226
Epoch 3/100
338/338 [==============================] - 16s 47ms/step - loss: 0.5604 - binary_accuracy: 0.7087 - val_loss: 0.5438 - val_binary_accuracy: 0.7176
Epoch 4/100
338/338 [==============================] - 16s 47ms/step - loss: 0.5570 - binary_accuracy: 0.7133 - val_loss: 0.5389 - val_binary_accuracy: 0.7255
Epoch 5/100
338/338 [==============================] - 16s 47ms/step - loss: 0.5514 - binary_accuracy: 0.7142 - val_loss: 0.5530 - val_binary_accuracy: 0.7065
Epoch 6/100
338/338 [==============================] - 16s 47ms/step - loss: 0.5580 - binary_accuracy: 0.7031 - val_loss: 0.5347 - val_binary_accuracy: 0.7300
Epoch 7/100
338/338 [=========================

###### Results:

- Title + Blurb + Story: CNN = 78.1% vs. 75.5%; RNN = 76.6% vs. 75.9%
- Title + Blurb + Story + Risks: CNN = 78.7% vs. 75.7%; RNN = 77.7% vs. 76.5%
- Title + Blurb + Story + Risks + Creator_Bio: CNN = 79.1% vs. 76.3%; RNN = 78% vs. 76.8%
- Title + Blurb + Story + Risks + Creator_Bio + Reward_Description: CNN = 82.9% vs. 76.5%; RNN = 78.2% vs. 77.1%
- i.e. using all text attributes helped to increase predictive performance

##### Fine-Tune Max_Len:

###### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

100%|██████████| 246891/246891 [00:42<00:00, 5780.91it/s]


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.200.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 200


In [ ]:
# Define parameters for text processing 
max_features = 631377
max_len = 300
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(len(word_index)))

Included Token: 631377


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,509.529984
std,409.239714
min,14.000000
25%,241.000000
50%,399.000000
75%,649.000000
max,11553.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 300)
Shape of X_val: (37034, 300)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (631377, 200)
Number of Null Word Embeddings: 218


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 300, 200)          126275400 
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 300, 64)           89664     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 60, 64)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 60, 64)            28736     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)               

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 16s 38ms/step - loss: 0.5992 - binary_accuracy: 0.6811 - val_loss: 0.5283 - val_binary_accuracy: 0.7330
Epoch 2/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5357 - binary_accuracy: 0.7276 - val_loss: 0.5407 - val_binary_accuracy: 0.7212
Epoch 3/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5173 - binary_accuracy: 0.7402 - val_loss: 0.5139 - val_binary_accuracy: 0.7433
Epoch 4/100
338/338 [==============================] - 9s 27ms/step - loss: 0.5066 - binary_accuracy: 0.7492 - val_loss: 0.5114 - val_binary_accuracy: 0.7426
Epoch 5/100
338/338 [==============================] - 9s 28ms/step - loss: 0.4960 - binary_accuracy: 0.7573 - val_loss: 0.4954 - val_binary_accuracy: 0.7569
Epoch 6/100
338/338 [==============================] - 9s 28ms/step - loss: 0.4848 - binary_accuracy: 0.7625 - val_loss: 0.5203 - val_binary_accuracy: 0.7472
Epoch 7/100
338/338 [==============================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 300, 200)          126275400 
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                29824     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 126,305,257
Trainable params: 29,857
Non-trainable params: 126,275,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 21s 50ms/step - loss: 0.6142 - binary_accuracy: 0.6607 - val_loss: 0.5861 - val_binary_accuracy: 0.7038
Epoch 2/100
338/338 [==============================] - 13s 39ms/step - loss: 0.5728 - binary_accuracy: 0.6997 - val_loss: 0.5471 - val_binary_accuracy: 0.7227
Epoch 3/100
338/338 [==============================] - 13s 39ms/step - loss: 0.5553 - binary_accuracy: 0.7194 - val_loss: 0.5525 - val_binary_accuracy: 0.7120
Epoch 4/100
338/338 [==============================] - 13s 39ms/step - loss: 0.5514 - binary_accuracy: 0.7214 - val_loss: 0.6184 - val_binary_accuracy: 0.6807
Epoch 5/100
338/338 [==============================] - 13s 39ms/step - loss: 0.5542 - binary_accuracy: 0.7094 - val_loss: 0.5422 - val_binary_accuracy: 0.7278
Epoch 6/100
338/338 [==============================] - 13s 39ms/step - loss: 0.5479 - binary_accuracy: 0.7163 - val_loss: 0.5731 - val_binary_accuracy: 0.6828
Epoch 7/100
338/338 [=========================

###### Results:

- max_len=300: CNN = 80.9% vs. 75.8%; RNN = 73.6% vs. 73.4%
- max_len=500: CNN = 82.9% vs. 76.5%; RNN = 78.2% vs. 77.1%
- max_len=700: CNN = 82.8% vs. 76.2%; RNN = 79% vs. 77.4%
- max_len=1000: CNN = 81% vs. 76.8%; RNN = 78.9% vs, 77.5%
- max_len=3000: CNN = 80.2% vs. 76.6%; RNN = takes too long to train
- i.e. max_len between 700 and 1000 provides the best trade-off between training time and accuracy

##### Fine-Tune Max_Features:

###### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

100%|██████████| 246891/246891 [00:42<00:00, 5753.81it/s]


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.200.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 200


In [ ]:
# Define parameters for text processing 
max_features = 50000
max_len = 700
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(tokenizer.num_words))

Included Token: 50000


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,495.489689
std,396.125344
min,14.000000
25%,235.000000
50%,388.000000
75%,631.000000
max,11407.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 700)
Shape of X_val: (37034, 700)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (50000, 200)
Number of Null Word Embeddings: 5


###### Convolutional Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(MaxPool1D(5))
  model.add(Conv1D(64, 7, activation='relu', padding='same'))
  model.add(GlobalMaxPool1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 700, 200)          10000000  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 700, 64)           89664     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 140, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 140, 64)           28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)               

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 18s 40ms/step - loss: 0.6009 - binary_accuracy: 0.6788 - val_loss: 0.5264 - val_binary_accuracy: 0.7364
Epoch 2/100
338/338 [==============================] - 8s 24ms/step - loss: 0.5355 - binary_accuracy: 0.7300 - val_loss: 0.5391 - val_binary_accuracy: 0.7206
Epoch 3/100
338/338 [==============================] - 8s 25ms/step - loss: 0.5160 - binary_accuracy: 0.7413 - val_loss: 0.5040 - val_binary_accuracy: 0.7506
Epoch 4/100
338/338 [==============================] - 8s 24ms/step - loss: 0.5028 - binary_accuracy: 0.7536 - val_loss: 0.5128 - val_binary_accuracy: 0.7447
Epoch 5/100
338/338 [==============================] - 8s 24ms/step - loss: 0.4919 - binary_accuracy: 0.7575 - val_loss: 0.4898 - val_binary_accuracy: 0.7611
Epoch 6/100
338/338 [==============================] - 8s 24ms/step - loss: 0.4793 - binary_accuracy: 0.7666 - val_loss: 0.4940 - val_binary_accuracy: 0.7572
Epoch 7/100
338/338 [==============================

###### Recurrent Neural Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(LSTM(32))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 700, 200)          10000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                29824     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 10,029,857
Trainable params: 29,857
Non-trainable params: 10,000,000
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 26s 64ms/step - loss: 0.6105 - binary_accuracy: 0.6597 - val_loss: 0.5527 - val_binary_accuracy: 0.7155
Epoch 2/100
338/338 [==============================] - 17s 52ms/step - loss: 0.5596 - binary_accuracy: 0.7111 - val_loss: 0.5535 - val_binary_accuracy: 0.7265
Epoch 3/100
338/338 [==============================] - 18s 53ms/step - loss: 0.5527 - binary_accuracy: 0.7193 - val_loss: 0.5677 - val_binary_accuracy: 0.6912
Epoch 4/100
338/338 [==============================] - 18s 52ms/step - loss: 0.5520 - binary_accuracy: 0.7195 - val_loss: 0.5328 - val_binary_accuracy: 0.7309
Epoch 5/100
338/338 [==============================] - 18s 52ms/step - loss: 0.5524 - binary_accuracy: 0.7131 - val_loss: 0.5436 - val_binary_accuracy: 0.7245
Epoch 6/100
338/338 [==============================] - 17s 52ms/step - loss: 0.5470 - binary_accuracy: 0.7102 - val_loss: 0.5432 - val_binary_accuracy: 0.7277
Epoch 7/100
338/338 [=========================

###### Results:

- max_features = None: CNN = 81% vs. 76.8%; RNN = 78.9% vs, 77.5%
- max_features = 200,000: CNN = 83.1% vs. 76.2%; RNN = 78.2% vs. 77.1%
- max_features = 100,000: CNN = 79.5% vs. 76.4%; RNN = 73.2% vs. 73.1%
- max_features = 50,000: CNN = 78.4% vs. 76.1%; RNN = 77% vs. 76.4%
- i.e. including all tokens helps to increase model performance (but also higher overfitting, which need to be addressed)

#### c) Fine-Tune CNN Model

##### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

100%|██████████| 246891/246891 [00:43<00:00, 5674.08it/s]


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.200.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 200


In [ ]:
# Define parameters for text processing 
max_features = 631377
max_len = 1000
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(len(word_index)))

Included Token: 631377


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,509.529984
std,409.239714
min,14.000000
25%,241.000000
50%,399.000000
75%,649.000000
max,11553.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 1000)
Shape of X_val: (37034, 1000)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (631377, 200)
Number of Null Word Embeddings: 218


##### Fine-Tune Network Architecture:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D(2))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D(2))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D(2))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D())
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_70"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_69 (Embedding)     (None, 1000, 200)         126275400 
_________________________________________________________________
conv1d_221 (Conv1D)          (None, 1000, 64)          64064     
_________________________________________________________________
average_pooling1d_52 (Averag (None, 500, 64)           0         
_________________________________________________________________
conv1d_222 (Conv1D)          (None, 500, 64)           20544     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 500, 64)           0         
_________________________________________________________________
average_pooling1d_53 (Averag (None, 250, 64)           0         
_________________________________________________________________
conv1d_223 (Conv1D)          (None, 250, 64)         

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 23s 52ms/step - loss: 0.5981 - binary_accuracy: 0.6652 - val_loss: 0.5252 - val_binary_accuracy: 0.7326
Epoch 2/100
338/338 [==============================] - 11s 31ms/step - loss: 0.5309 - binary_accuracy: 0.7275 - val_loss: 0.5120 - val_binary_accuracy: 0.7384
Epoch 3/100
338/338 [==============================] - 11s 32ms/step - loss: 0.5158 - binary_accuracy: 0.7402 - val_loss: 0.5153 - val_binary_accuracy: 0.7355
Epoch 4/100
338/338 [==============================] - 11s 32ms/step - loss: 0.5087 - binary_accuracy: 0.7467 - val_loss: 0.4932 - val_binary_accuracy: 0.7560
Epoch 5/100
338/338 [==============================] - 11s 31ms/step - loss: 0.5035 - binary_accuracy: 0.7491 - val_loss: 0.5099 - val_binary_accuracy: 0.7426
Epoch 6/100
338/338 [==============================] - 11s 33ms/step - loss: 0.4947 - binary_accuracy: 0.7544 - val_loss: 0.5264 - val_binary_accuracy: 0.7412
Epoch 7/100
338/338 [=========================

Result:
- 1 CNN Layer, 1 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 2: 77.9% vs. 74.6%
- 1 CNN Layer, 1 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 5: 79.5% vs. 75%
- 1 CNN Layer, 1 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 10: 82% vs. 75.4%
- 1 CNN Layer, 1 Dense Layers, 32 Filters, Kernel Size 7, Pooling Size 10: 77.3% vs. 75.3%
- 1 CNN Layer, 1 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10: 78.7% vs. 75.5%
-> i.e. the less parameters before the output layer, the less overfitting
- 1 CNN Layer, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons 32: 79.6% vs. 76.1%
- 1 CNN Layer, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons 64: 76.9% vs. 75.7%
- 1 CNN Layer, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons 16: 78.3% vs. 75.9%
- 1 CNN Layer, 3 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons [64,16]: 78.8% vs. 75.9%
- 1 CNN Layer, 3 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons [32,8]: 78.1% vs. 75.8%
-> i.e. adding an additional hidden Dense layer helps to improve performance (stick first with 1 hidden layer)
- 2 CNN Layers, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons 32: 78.2% vs. 76.2% 
- 2 CNN Layers, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons 16: 79.1% vs. 76%
- 2 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 10, Dense Neurons 32: 79.1% vs. 76.5%
- 2 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 5, Dense Neurons 32: 79.4% vs. 76.5%
- 2 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 79.7% vs. 76.7%
- 2 CNN Layers, 2 Dense Layers, 32 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 78.7% vs. 76.6%
-> i.e. adding an additional CNN layer helps
- 3 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 82.7% vs. 76.6%
- 3 CNN Layers, 2 Dense Layers, 32 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 79.8% vs. 76.9%
- 3 CNN Layers, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 77.5% vs. 76.5%
- 4 CNN Layers, 2 Dense Layers, 32 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 80% vs. 77%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 79% vs. 77.1%
- 4 CNN Layers, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 78.6% vs. 77%
- 5 CNN Layers, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 78.7% vs. 76.7%
- 5 CNN Layers, 2 Dense Layers, 32 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 78.9% vs. 77%
- 5 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 80.1% vs. 76.9%
- 6 CNN Layers, 2 Dense Layers, 32 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 80.5% vs. 76.8%
- 6 CNN Layers, 2 Dense Layers, 16 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 78.9% vs. 76.6%
-> i.e. 4 CNN layers worked the best; 32-64 filters worked the best; pooling of 2 worked the best
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 3, Pooling Size 2, Dense Neurons 32: 77.5% vs. 76.6%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32: 80.6% vs. 77.4%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 7, Pooling Size 2, Dense Neurons 32: 79% vs. 77.1%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 10, Pooling Size 2, Dense Neurons 32: 78.5% vs. 76.7%
-> i.e. kernel size of 5 performs the best
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32: 80.6% vs. 77.4%
- 4 CNN Layers, 2 Dense Layers, 32 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32: 77.2% vs. 76.6%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 64: 78% vs. 76.6%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 16: 80.9% vs. 77.2%
- 4 CNN Layers, 3 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons [64,16]: 83.5% vs. 76.5%
- 4 CNN Layers, 3 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons [32,32]: 80.2% vs. 77.4%
- 4 CNN Layers, 3 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons [64,64]: 78.2% vs. 76.9%
- 4 CNN Layers, 3 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons [16,16]: 80.8% vs. 77.2%
-> i.e. 2 Dense Layers with 32 hidden units provided best results
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, padding=same: 80.6% vs. 77.4%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, padding=valid: 79.3% vs. 77.2%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, padding=causal: 80.3% vs. 77.2%
-> i.e. same padding is a little bit better
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=max, flatten: 80.6% vs. 77.4%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten: 80.1% vs. 77.5%
- 4 CNN Layers, 1 Dense Layer, 64 Filters, Kernel Size 5, Pooling Size 2, pooling=avg, global: 80.9% vs. 77.1%
- 4 CNN Layers, 1 Dense Layer, 64 Filters, Kernel Size 5, Pooling Size 2, pooling=max, global: 79.5% vs. 76.5%
-> i.e. average pooling with flatten provided the best results
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, relu: 80.4% vs. 77.4%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, elu: 76.5% vs. 75.1%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, selu: 75.3% vs. 73.5%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, gelu: 76.5% vs. 75.6%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, swish: 79% vs. 75.9%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, tanh: 78% vs. 74.5%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, sigmoid: 74.8% vs. 74.6%
- 4 CNN Layers, 2 Dense Layers, 64 Filters, Kernel Size 5, Pooling Size 2, Dense Neurons 32, pooling=avg, flatten, leaky relu: 78.7% vs. 76%
-> i.e. relu activation was the best

##### Add Regularization To The Network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same'))
  model.add(AvgPool1D())
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 200)         126275400 
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1000, 64)          64064     
_________________________________________________________________
average_pooling1d_4 (Average (None, 500, 64)           0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 500, 64)           0         
_________________________________________________________________
layer_normalization_5 (Layer (None, 500, 64)           128       
_________________________________________________________________
dropout_5 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 500, 64)          

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=10, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 28s 60ms/step - loss: 2.1265 - binary_accuracy: 0.6202 - val_loss: 0.6253 - val_binary_accuracy: 0.6767
Epoch 2/100
338/338 [==============================] - 11s 33ms/step - loss: 0.6282 - binary_accuracy: 0.6766 - val_loss: 0.5976 - val_binary_accuracy: 0.6986
Epoch 3/100
338/338 [==============================] - 12s 34ms/step - loss: 0.6206 - binary_accuracy: 0.6864 - val_loss: 0.6131 - val_binary_accuracy: 0.6825
Epoch 4/100
338/338 [==============================] - 12s 34ms/step - loss: 0.6171 - binary_accuracy: 0.6918 - val_loss: 0.6744 - val_binary_accuracy: 0.5937
Epoch 5/100
338/338 [==============================] - 11s 34ms/step - loss: 0.6178 - binary_accuracy: 0.6889 - val_loss: 0.5875 - val_binary_accuracy: 0.7204
Epoch 6/100
338/338 [==============================] - 12s 34ms/step - loss: 0.6187 - binary_accuracy: 0.6932 - val_loss: 0.5889 - val_binary_accuracy: 0.7190
Epoch 7/100
338/338 [=========================

Result:
- None: 80.4% vs. 77.4%
- Dropout: 80.5% vs. 77.8% (Input=0.2, Layers=0.5)
- Batch Normalization: led to extreme overfitting; 81.4% vs. 77.6% (with dropout)
- Layer Normalization: 81.7% vs. 78.2% (first normalization, then dropout, no input dropout)
- Gaussian Noise: 79.9% vs. 78.2% (incl. normalization + dropout)
- L2: 72.5% vs. 72.5%
-> i.e. a combination of Gaussian Noise + Layer Normalization + Dropout provided the best results

##### Fine-Tune Optimization Algorithm:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D())
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Flatten())
  model.add(Dense(32, activation='relu', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(seed=seed_value)))
  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1000, 200)         126275400 
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 1000, 64)          64064     
_________________________________________________________________
average_pooling1d_52 (Averag (None, 500, 64)           0         
_________________________________________________________________
gaussian_noise_61 (GaussianN (None, 500, 64)           0         
_________________________________________________________________
layer_normalization_65 (Laye (None, 500, 64)           128       
_________________________________________________________________
dropout_65 (Dropout)         (None, 500, 64)           0         
_________________________________________________________________
conv1d_53 (Conv1D)           (None, 500, 64)         

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=10, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
169/169 [==============================] - 27s 94ms/step - loss: 0.7201 - binary_accuracy: 0.6028 - val_loss: 0.5533 - val_binary_accuracy: 0.7007
Epoch 2/100
169/169 [==============================] - 6s 38ms/step - loss: 0.5609 - binary_accuracy: 0.6988 - val_loss: 0.5332 - val_binary_accuracy: 0.7234
Epoch 3/100
169/169 [==============================] - 6s 38ms/step - loss: 0.5402 - binary_accuracy: 0.7202 - val_loss: 0.5233 - val_binary_accuracy: 0.7327
Epoch 4/100
169/169 [==============================] - 6s 38ms/step - loss: 0.5335 - binary_accuracy: 0.7275 - val_loss: 0.5096 - val_binary_accuracy: 0.7419
Epoch 5/100
169/169 [==============================] - 6s 38ms/step - loss: 0.5257 - binary_accuracy: 0.7345 - val_loss: 0.5089 - val_binary_accuracy: 0.7421
Epoch 6/100
169/169 [==============================] - 6s 38ms/step - loss: 0.5196 - binary_accuracy: 0.7387 - val_loss: 0.5213 - val_binary_accuracy: 0.7330
Epoch 7/100
169/169 [==============================

Results:
- RMSprop: 79.9% vs. 78.2%
- RMSprop_centered: 79.1% vs. 78.1%
- Adam: 80.1% vs. 78.2%
- Adam_amsgrad: not supported with TPU
- Adamax: 79.5% vs. 78.2% (but took extremely long)
- Nadam: 79.8% vs. 78.3%
-> i.e. Nadam provided the best results
- GlorotUniform: 79.8% vs. 78.3%
- GlorotNormal: 80.3% vs. 78.2%
- HeUniform: 80.1% vs. 78.3%
- HeNormal: 80.9% vs. 78.2%
- LecunUniform: 80.8% vs. 78.3%
- LecunNormal: 80% vs. 78.1%
-> i.e. GlorotUniform provided the best results
- But: differences were only marginal between all optimizers and initializers

##### Try different types of CNN layers:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D())
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Flatten())
  model.add(Dense(32, activation='relu', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(seed=seed_value)))
  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 1000, 200)         126275400 
_________________________________________________________________
locally_connected1d_1 (Local (None, 996, 64)           63807744  
_________________________________________________________________
average_pooling1d_60 (Averag (None, 498, 64)           0         
_________________________________________________________________
gaussian_noise_71 (GaussianN (None, 498, 64)           0         
_________________________________________________________________
layer_normalization_75 (Laye (None, 498, 64)           128       
_________________________________________________________________
dropout_75 (Dropout)         (None, 498, 64)           0         
_________________________________________________________________
locally_connected1d_2 (Local (None, 494, 64)         

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=10, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Results:
- CNN: 79.8% vs. 78.3%
- Separable CNN: 80% vs. 78.1%
- Locally-Connected CNN: overfits quite badly

#### d) Fine-Tune RNN Model

##### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

100%|██████████| 246891/246891 [00:43<00:00, 5664.08it/s]


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
import fasttext, fasttext.util
ft = fasttext.load_model('cc.en.300.bin')
print("Embedding Dimension: {}".format(ft.get_dimension()))

Embedding Dimension: 200


In [ ]:
# Define parameters for text processing 
max_features = 631377
max_len = 1000
embedding_dim = 300

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(len(word_index)))

Included Token: 631377


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,509.529984
std,409.239714
min,14.000000
25%,241.000000
50%,399.000000
75%,649.000000
max,11553.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 1000)
Shape of X_val: (37034, 1000)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      embedding_vector = ft.get_word_vector(word)
      if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (631377, 300)
Number of Null Word Embeddings: 218


##### Test LSTM vs GRU:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(GRU(32, kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 1000, 200)         126275400 
_________________________________________________________________
gru (GRU)                    (None, 32)                22464     
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 33        
Total params: 126,297,897
Trainable params: 22,497
Non-trainable params: 126,275,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=10, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 29s 73ms/step - loss: 0.6205 - binary_accuracy: 0.6550 - val_loss: 0.5700 - val_binary_accuracy: 0.6860
Epoch 2/100
338/338 [==============================] - 19s 57ms/step - loss: 0.5688 - binary_accuracy: 0.6899 - val_loss: 0.5612 - val_binary_accuracy: 0.7090
Epoch 3/100
338/338 [==============================] - 19s 56ms/step - loss: 0.5448 - binary_accuracy: 0.7172 - val_loss: 0.5262 - val_binary_accuracy: 0.7356
Epoch 4/100
338/338 [==============================] - 19s 57ms/step - loss: 0.5261 - binary_accuracy: 0.7343 - val_loss: 0.5222 - val_binary_accuracy: 0.7351
Epoch 5/100
338/338 [==============================] - 19s 56ms/step - loss: 0.5192 - binary_accuracy: 0.7374 - val_loss: 0.5119 - val_binary_accuracy: 0.7420
Epoch 6/100
338/338 [==============================] - 19s 57ms/step - loss: 0.5113 - binary_accuracy: 0.7439 - val_loss: 0.5060 - val_binary_accuracy: 0.7490
Epoch 7/100
338/338 [=========================

Result: 
- LSTM: 79.1% vs. 77.3%
- GRU: 79.3% vs. 77.7%
-> i.e. GRU was better and converged faster

##### Test Regularization Techniques Before Dense Layer and After Input Layer:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False))
  model.add(GaussianNoise(stddev=0.1))
  model.add(GRU(64, kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1000, 200)         126275400 
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1000, 200)         0         
_________________________________________________________________
gru_12 (GRU)                 (None, 64)                51072     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 126,326,537
Trainable params: 51,137
Non-trainable params: 126,275,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=10, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Results:
- Dropout in RNN = takes too long to train
- Dropout Before Dense = 80% vs. 78.2% (Dense=0.5)
- Dropout Before Dense + After Input: 80.5% vs. 78.3%; but took longer to train (Input=0.2, Dense=0.5)
- Batch Normalization Before Dense: was not useful; accuracy jumped around
- Layer Normalization Before Dense: 79.1% vs. 77.6%
- Layer Normalization + Dropout Before Dense: 79.9% vs. 78.1%
- Gaussian Noise Before Dense: 79.2% vs. 78%
-> i.e. adding only dropout provided the best results; maybe now it is possible to increase the capacity of the network

##### Fine-Tune Network Architecture:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False))
  model.add(Bidirectional(GRU(64, kernel_initializer=GlorotUniform(seed=seed_value))))
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"], steps_per_execution=100)

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 1000, 200)         126275400 
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               102144    
_________________________________________________________________
dropout_31 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 129       
Total params: 126,377,673
Trainable params: 102,273
Non-trainable params: 126,275,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 41s 120ms/step - loss: 0.6251 - binary_accuracy: 0.6573 - val_loss: 0.5656 - val_binary_accuracy: 0.6889
Epoch 2/100
338/338 [==============================] - 22s 65ms/step - loss: 0.5488 - binary_accuracy: 0.7144 - val_loss: 0.5415 - val_binary_accuracy: 0.7157
Epoch 3/100
338/338 [==============================] - 22s 65ms/step - loss: 0.5197 - binary_accuracy: 0.7392 - val_loss: 0.5062 - val_binary_accuracy: 0.7462
Epoch 4/100
338/338 [==============================] - 22s 65ms/step - loss: 0.5123 - binary_accuracy: 0.7459 - val_loss: 0.5007 - val_binary_accuracy: 0.7520
Epoch 5/100
338/338 [==============================] - 22s 65ms/step - loss: 0.5068 - binary_accuracy: 0.7477 - val_loss: 0.4984 - val_binary_accuracy: 0.7543
Epoch 6/100
338/338 [==============================] - 22s 65ms/step - loss: 0.4992 - binary_accuracy: 0.7554 - val_loss: 0.5090 - val_binary_accuracy: 0.7531
Epoch 7/100
338/338 [========================

Results:
- 512 Units: 79.2% vs. 77.5%
- 256 Units: 81.6% vs. 77.7%
- 128 Units: 80.5% vs. 78%
- 64 Units: 80% vs. 78.2%
- 32 Units: 79.9% vs. 78%
- 2 Layers, 32 Units: 79.2% vs. 77.9%
- 2 Layers, 64 Units: 79.8% vs. 77.8%
- Bidirectional 64 Units: 80.5% vs. 77.6%
-> i.e. 1 Layer, 64 Units is the best


##### Test Recurrent Dropout:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False))
  model.add(GRU(256, kernel_initializer=GlorotUniform(seed=seed_value), dropout=0.5, recurrent_dropout=0.5))
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(clipnorm=1.0), loss="binary_crossentropy", metrics=["binary_accuracy"], steps_per_execution=338)

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 200)         126275400 
_________________________________________________________________
gru (GRU)                    (None, 256)               351744    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 126,627,401
Trainable params: 352,001
Non-trainable params: 126,275,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 50s 147ms/step - loss: 0.6295 - binary_accuracy: 0.6585 - val_loss: 0.6033 - val_binary_accuracy: 0.7090
Epoch 2/100
338/338 [==============================] - 32s 96ms/step - loss: 0.5466 - binary_accuracy: 0.7169 - val_loss: 0.5633 - val_binary_accuracy: 0.6992
Epoch 3/100
338/338 [==============================] - 32s 95ms/step - loss: 0.5279 - binary_accuracy: 0.7318 - val_loss: 0.5099 - val_binary_accuracy: 0.7430
Epoch 4/100
338/338 [==============================] - 32s 95ms/step - loss: 0.5186 - binary_accuracy: 0.7378 - val_loss: 0.5005 - val_binary_accuracy: 0.7495
Epoch 5/100
338/338 [==============================] - 32s 95ms/step - loss: 0.5115 - binary_accuracy: 0.7433 - val_loss: 0.4955 - val_binary_accuracy: 0.7528
Epoch 6/100
338/338 [==============================] - 32s 95ms/step - loss: 0.5049 - binary_accuracy: 0.7480 - val_loss: 0.4998 - val_binary_accuracy: 0.7494
Epoch 7/100
338/338 [========================

Results:
- dropout=0.5, recurrent_dropout=0.5, 32 Units: underfits too much
- dropout=0.5, recurrent_dropout=0.5, 64 Units: 80.3% vs. 78.6%
- dropout=0.5, recurrent_dropout=0.5, 128 Units: 79.9% vs. 78.4%
- dropout=0.5, recurrent_dropout=0.5, 256 Units: 82.2% vs. 78.8%
- dropout=0.5, recurrent_dropout=0.5, 512 Units: jumps around to heavily
- dropout=0.5, recurrent_dropout=0.5, 64 Units, 2 Layers: 80.3% vs. 78.7%
- dropout=0.5, recurrent_dropout=0.5, 256 Units, 2 Layers: 82.1% vs. 78.8%
- dropout=0.5, recurrent_dropout=0.5, 64 Units, 3 Layers: 80.2% vs. 78.7%
-> i.e. dropout + 1 layer + 256 units provided the best results
-> adding additional layers did not help to improve performance
- dropout=0.3, recurrent_dropout=0.3, 64 Units: 79.4% vs. 78.4%
- dropout=0.3, recurrent_dropout=0.3, 256 Units: 80.4% vs.78.4%
-> i.e. a high dropout of 0.5 provided the best results
- dropout=0.5, recurrent_dropout=0.5, dense_dropout=0.5, 256 Units: 81.4% vs. 78.8%
-> i.e. adding Dense dropout helps a little bit to reduce overfitting
- dropout=0.5, recurrent_dropout=0.5, dense_dropout=0.5, 256 Units; gradient-clipping: 80.9% vs. 78.8%
-> i.e. gradient clipping further reduced the overfitting effect

##### Re-try 300-dimensional embeddings with the regularized network:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False))
  model.add(GRU(256, kernel_initializer=GlorotUniform(seed=seed_value), dropout=0.5, recurrent_dropout=0.5))
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(clipnorm=1.0), loss="binary_crossentropy", metrics=["binary_accuracy"], steps_per_execution=338)

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 300)         189413100 
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               428544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 189,841,901
Trainable params: 428,801
Non-trainable params: 189,413,100
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

338/338 [==============================] - 56s 164ms/step - loss: 0.6206 - binary_accuracy: 0.6597 - val_loss: 0.6063 - val_binary_accuracy: 0.6621
Epoch 2/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5427 - binary_accuracy: 0.7203 - val_loss: 0.5397 - val_binary_accuracy: 0.7083
Epoch 3/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5220 - binary_accuracy: 0.7353 - val_loss: 0.5046 - val_binary_accuracy: 0.7487
Epoch 4/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5123 - binary_accuracy: 0.7430 - val_loss: 0.4958 - val_binary_accuracy: 0.7541
Epoch 5/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5043 - binary_accuracy: 0.7487 - val_loss: 0.4901 - val_binary_accuracy: 0.7582
Epoch 6/100
338/338 [==============================] - 36s 105ms/step - loss: 0.4972 - binary_accuracy: 0.7536 - val_loss: 0.4897 - val_binary_accuracy: 0.7572
Epoch 7/100
338/338 [==============================]

Result:
- 81.9% vs. 78.9%
-> i.e. using 300-dimensional embeddings minimally increased model performance
-> since 300-dimensional embeddings also worked better with CNNs, switch back again to 300 dimensions

#### e) Try Using different embeddings

##### Data Preparation:

In [ ]:
# Extract Target Variable From Dataset
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]

print(type(y))
print(y.shape)
print(type(text))
print(len(text))

<class 'numpy.ndarray'>
(246891,)
<class 'pandas.core.series.Series'>
246891


In [ ]:
# Remove stopwords and punctation
stopwords = set(list(ENGLISH_STOP_WORDS) + ["s"])
text = text.str.replace(r"\.", "")
text = text.progress_apply(lambda x: " ".join([token for token in x.split() if token not in stopwords]))
text = text.to_numpy()

In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(len(text_train)))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(len(text_subtrain)))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(len(text_val)))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(len(text_test)))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: 209858
Shape of y_train: (209858,)
Shape of X_subtrain: 172824
Shape of y_subtrain: (172824,)
Shape of X_val: 37034
Shape of y_val: (37034,)
Shape of X_test: 37033
Shape of y_test: (37033,)


In [ ]:
# Download word embeddings
import gensim.downloader as api
glove = api.load("glove-twitter-200")

In [ ]:
# Define parameters for text processing 
max_features = 100000
max_len = 1000
embedding_dim = 200

In [ ]:
# Convert Texts Into Integer Sequences (Tokenization)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(text_subtrain)
X_subtrain = tokenizer.texts_to_sequences(text_subtrain)
X_val = tokenizer.texts_to_sequences(text_val)
word_index = tokenizer.word_index
print("Included Token: {}".format(tokenizer.num_words))

Included Token: 100000


In [ ]:
# Determine number of words for max_len
num_words = pd.DataFrame([len(x) for x in X_subtrain])
num_words.rename(columns={0:"words"}, inplace=True)
num_words.describe()

,words
count,172824.000000
mean,502.025529
std,402.349536
min,14.000000
25%,237.000000
50%,393.000000
75%,640.000000
max,11478.000000


In [ ]:
# Pad and Truncate sequences
X_subtrain = pad_sequences(X_subtrain, maxlen=max_len, padding="pre", truncating="post")
X_val = pad_sequences(X_val, maxlen=max_len, padding="pre", truncating="post")

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))

Shape of X_subtrain: (172824, 1000)
Shape of X_val: (37034, 1000)


In [ ]:
# Prepare the Embedding Matrix
print('Preparing Embedding Matrix...')
words_not_found = []
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
      if word in glove.vocab:
        embedding_matrix[i] = glove.get_vector(word)
      else:
        words_not_found.append(word)
print('Shape of Embedding Matrix: {}'.format(embedding_matrix.shape))
print('Number of Null Word Embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing Embedding Matrix...
Shape of Embedding Matrix: (100000, 300)
Number of Null Word Embeddings: 41096


##### CNN:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D(2))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Conv1D(64, 5, activation='relu', padding='same', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(AvgPool1D())
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Flatten())
  model.add(Dense(32, activation='relu', kernel_initializer=GlorotUniform(seed=seed_value)))
  model.add(GaussianNoise(stddev=0.1))
  model.add(LayerNormalization())
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(seed=seed_value)))
  return model

In [ ]:
# Compile the model
with strategy.scope():
  model = create_model()
  model.compile(optimizer=Nadam(), loss="binary_crossentropy", metrics=["binary_accuracy"])

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 1000, 300)         30000000  
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 1000, 64)          96064     
_________________________________________________________________
average_pooling1d_27 (Averag (None, 500, 64)           0         
_________________________________________________________________
gaussian_noise_27 (GaussianN (None, 500, 64)           0         
_________________________________________________________________
layer_normalization_29 (Laye (None, 500, 64)           128       
_________________________________________________________________
dropout_42 (Dropout)         (None, 500, 64)           0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 500, 64)         

In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 32s 61ms/step - loss: 0.6690 - binary_accuracy: 0.6314 - val_loss: 0.5400 - val_binary_accuracy: 0.7193
Epoch 2/100
338/338 [==============================] - 10s 30ms/step - loss: 0.5441 - binary_accuracy: 0.7163 - val_loss: 0.5566 - val_binary_accuracy: 0.6946
Epoch 3/100
338/338 [==============================] - 10s 31ms/step - loss: 0.5266 - binary_accuracy: 0.7328 - val_loss: 0.5050 - val_binary_accuracy: 0.7459
Epoch 4/100
338/338 [==============================] - 10s 31ms/step - loss: 0.5175 - binary_accuracy: 0.7420 - val_loss: 0.5067 - val_binary_accuracy: 0.7493
Epoch 5/100
338/338 [==============================] - 11s 31ms/step - loss: 0.5098 - binary_accuracy: 0.7470 - val_loss: 0.4955 - val_binary_accuracy: 0.7551
Epoch 6/100
338/338 [==============================] - 11s 32ms/step - loss: 0.4993 - binary_accuracy: 0.7534 - val_loss: 0.4959 - val_binary_accuracy: 0.7534
Epoch 7/100
338/338 [=========================

##### RNN:

In [ ]:
# Define the model
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=max_len, trainable=False))
  model.add(GRU(256, kernel_initializer=GlorotUniform(seed=seed_value), dropout=0.5, recurrent_dropout=0.5))
  model.add(Dropout(rate=0.5, seed=seed_value))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
# Compile the model
with strategy.scope():
    model = create_model()
    model.compile(optimizer=Nadam(clipnorm=1.0), loss="binary_crossentropy", metrics=["binary_accuracy"], steps_per_execution=338)

In [ ]:
# Show summary of the model
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 1000, 300)         30000000  
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               428544    
_________________________________________________________________
dropout_47 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 257       
Total params: 30,428,801
Trainable params: 428,801
Non-trainable params: 30,000,000
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=7, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 53s 158ms/step - loss: 0.6043 - binary_accuracy: 0.6685 - val_loss: 0.5945 - val_binary_accuracy: 0.7197
Epoch 2/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5357 - binary_accuracy: 0.7252 - val_loss: 0.5485 - val_binary_accuracy: 0.7012
Epoch 3/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5140 - binary_accuracy: 0.7410 - val_loss: 0.4971 - val_binary_accuracy: 0.7525
Epoch 4/100
338/338 [==============================] - 36s 105ms/step - loss: 0.5012 - binary_accuracy: 0.7504 - val_loss: 0.4822 - val_binary_accuracy: 0.7632
Epoch 5/100
338/338 [==============================] - 36s 105ms/step - loss: 0.4918 - binary_accuracy: 0.7581 - val_loss: 0.4770 - val_binary_accuracy: 0.7674
Epoch 6/100
338/338 [==============================] - 36s 105ms/step - loss: 0.4819 - binary_accuracy: 0.7651 - val_loss: 0.4719 - val_binary_accuracy: 0.7719
Epoch 7/100
338/338 [===================

##### Results:

- FastText 300d: CNN = 79.8% vs. 78.3%; RNN = 81.9% vs. 78.9%
- Glove Twitter 200d: 82.1% vs. 78.2%; RNN = 82% vs. 78.9%
- Glove Wikipedia + Gigaword 300d: CNN = 80.8% vs. 78.3%; RNN = 80.3% vs. 78.4%
- Word2Vec Google News 300d: CNN = 83.4% vs. 78.4%; RNN = 82% vs. 78.9%
- i.e. different embeddings did not make a huge difference
- i.e. stick to FastText, as it provided the best trade-off between good accuracy and low overfitting

#### f) Best-Found Model: NN Text

- Preprocessing: Stopwords Removed; All Text Attributes; No max_features restrictions; max_len=1000; embedding dimension = 300; Fasttext
- CNN: 4 Conv Layer (64 Filter, Window Size 5, ReLU, padding=same); AveragePooling (Size 2); Gaussian Noise; Layer Normalization; Dropout (Rate=0.5); 1 Dense Layer (32 Units)
- RNN: 1 GRU Layer (256 Units); Input/Output/Recurrent Dropout (Rate=0.5)
- Optimization: GlorotUniform; Nadam; clipnorm=1.0; Batch Size: 512